# Runnable PromptTemplates
[![Open In Collab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langsmith-cookbook/blob/main/hub-examples/runnable-prompt/edit-in-playground.ipynb)

When you build a chain using LangChain [runnables](https://python.langchain.com/docs/expression_language), each component is assigned its own run span.

This means you can modify and run prompts directly in the playground. You can also save and version them in the hub for later use.

In this example, you will build a simple chain using runnables, save the prompt trace to the hub, and then use the versioned prompt within your chain. The prompt will look something like the following:

[![Chat Prompt Repo](https://github.com/langchain-ai/langsmith-cookbook/blob/main/hub-examples/runnable-prompt/img/prompt_repo.png?raw=1)](https://smith.langchain.com/hub/wfh/enigma-prompt)

Using runnables in the playground make it easy to experiment quickly with different prompts and to share them with your team, especially for those who prefer to not dive deep into the code.

## Prerequisites

Before you start, make sure you have a LangSmith account and an API key (for your personal "organization"). For more information on getting started, check out the [docs](https://docs.smith.langchain.com/hub/quickstart).

In [1]:
%pip install -U langchain langchainhub  --quiet

In [2]:
%pip install --upgrade anthropic

In [3]:
import os

from google.colab import userdata

# Update with your API URL if using a hosted instance of Langsmith.
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')s
# Update with your API URL if using a hosted instance of Langsmith.
os.environ["LANGCHAIN_HUB_API_URL"] = "https://api.hub.langchain.com"
os.environ["LANGCHAIN_HUB_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["ANTHROPIC_API_KEY"] = userdata.get('ANTHROPIC_API_KEY')

## 1. Define your chain

First, define the chain you want to trace. We will start with a simple prompt and chat-model combination.


In [6]:
from langchain import prompts, hub
import anthropic
from langchain_community.chat_models import ChatAnthropic
from langchain_community.chat_models import ChatOpenAI
from langchain_community.embeddings import OpenAIEmbeddings


prompt = prompts.ChatPromptTemplate.from_messages(
    [
        ("system", "You are a mysteriously vague oracle who only speaks in riddles."),
        ("human", "{input}"),
    ]
)

chain = prompt | ChatOpenAI(model_name="gpt-3.5-turbo", temperature=1)

<ipython-input-6-b6c3953cc8c5>:15: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chain = prompt | ChatOpenAI(model_name="gpt-3.5-turbo", temperature=1)


The pipe "|" syntax above converts the prompt and chat model into a [RunnableSequence](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.base.RunnableSequence.html) for easy composition.

Run the chain below. We will use the [collect_runs](https://api.python.langchain.com/en/latest/tracers/langchain_core.tracers.context.collect_runs.html) callback to retrieve the run ID of the traced run and share it.

In [7]:
from langchain import callbacks

with callbacks.collect_runs() as cb:
    for chunk in chain.stream({"input": "What is the GIL?"}):
        print(chunk.content, end="", flush=True)
    run_id = cb.traced_runs[0].id

The serpent coils tightly around the heart of the machine, binding its power in a dance of limitation and concurrency.